In [95]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [96]:
# loading the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

In [97]:
# splitting the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [98]:
# class for a node in the decision tree
class Node:
    # constructor for the node class
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None, parent = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.parent = parent
    # check if node is a leaf node
    def check_if_leaf_leaf(self):
        if (self.left == None and self.right == None):
            return True
        return False
    # check if node is a root node
    def check_if_root(self):
        if (self.parent == None):
            return True
        return False
    # check if node is a left child
    def check_if_left_child(self):
        if (self.parent.left == self):
            return True
        return False
    # check if node is a right child
    def check_if_right_child(self):
        if (self.parent.right == self):
            return True
        return False
    # check if node is a parent
    def check_if_parent(self):
        if (self.left != None or self.right != None):
            return True
        return False

In [99]:
# defining the decision tree class for classification
class DecisionTree:
    # defining the constructor
    def __init__(self, no_of_minimum_sample_splits = 2, maximum_depth = 1000):
        self.no_of_minimum_sample_splits = no_of_minimum_sample_splits
        self.maximum_depth = maximum_depth
    # defining the function to compute the gini index
    def compute_gini_index(self, y):
        classes = np.unique(y, return_counts=True)
        impurity = 1
        for i in classes[1]:
            impurity -= (i/len(y))**2
        return impurity
    # defining the function to compute the information gain
    def compute_information_gain(self, y, y1, y2):
        # information gain = entropy(parent) - [weighted average]entropy(children)
        left_weight = len(y1)/len(y)
        right_weight = len(y2)/len(y)
        information_gain = self.compute_gini_index(y) - (left_weight*self.compute_gini_index(y1) + right_weight*self.compute_gini_index(y2))
        return information_gain
    # defining the recursive function to build the decision tree
    def build_tree(self, X, y, depth=0):
        sample_num = X.shape[0]
        feature_num = X.shape[1]
        class_num = len(np.unique(y))
        # if the number of samples is greater than the minimum sample split and the depth is less than the maximum depth, then the tree is not built further
        if sample_num >= self.no_of_minimum_sample_splits and depth <= self.maximum_depth:
            best_information_gain = 0
            best_feature = None
            best_threshold = None
            for i in range(feature_num):
                thresholds = np.unique(X[:, i])
                for j in thresholds:
                    y_left = y[X[:, i] <= j]
                    y_right = y[X[:, i] > j]
                    # if the left or the right size is empty, then it means that the threshold is not valid
                    if len(y_left) == 0 or len(y_right) == 0:
                        continue
                    else:
                        # computing the information gain
                        information_gain = self.compute_information_gain(y, y_left, y_right)
                        # if the information gain is greater than the best information gain, then the best information gain is updated
                        if information_gain > best_information_gain:
                            best_information_gain = information_gain
                            best_feature = i
                            best_threshold = j
            # if the best feature is not None, then the tree is built further
            if best_feature != None:
                left_indices = np.argwhere(X[:, best_feature] <= best_threshold).flatten()
                right_indices = np.argwhere(X[:, best_feature] > best_threshold).flatten()
                left = self.build_tree(X[left_indices, :], y[left_indices], depth+1)
                right = self.build_tree(X[right_indices, :], y[right_indices], depth+1)
                return Node(best_feature, best_threshold, left, right)
        # if the number of samples is less than the minimum sample split or the depth is greater than the maximum depth, then the leaf node is returned
        leaf_value = np.argmax(np.bincount(y))
        return Node(value=leaf_value)
    # defining the function to fit the decision tree
    def train(self, X, y):
        self.root = self.build_tree(X, y)
    # defining the function to predict the output
    def predict_output(self, x, node):
        if node.check_if_leaf_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.predict_output(x, node.left)
        return self.predict_output(x, node.right)
    def predict(self, X):
        y_pred = []
        for x in X:
            y_pred.append(self.predict_output(x, self.root))
        return np.array(y_pred)
    

In [100]:
# defining the decision tree 
decision_tree = DecisionTree()
# fitting the decision tree
decision_tree.train(X_train, y_train)
# predicting the output
y_pred = decision_tree.predict(X_test)
# printing the accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9333333333333333


In [101]:
# printing the decision tree
num = 1
def print_decision_tree(node, depth=0):
    global num
    if node.check_if_leaf_leaf():
        global num
        print(depth*"\t"+str(num)+" Leaf Node:", node.value, "Class:", iris.target_names[node.value])
        num = num+1
        return
    print(depth*"\t"+str(num)+" Feature:", iris.feature_names[node.feature], "Threshold:", node.threshold)
    num = num+1
    print_decision_tree(node.left, depth+1)
    print_decision_tree(node.right, depth+1)
print_decision_tree(decision_tree.root)

1 Feature: petal length (cm) Threshold: 1.7
	2 Leaf Node: 0 Class: setosa
	3 Feature: petal width (cm) Threshold: 1.7
		4 Feature: petal length (cm) Threshold: 4.9
			5 Feature: petal width (cm) Threshold: 1.6
				6 Leaf Node: 1 Class: versicolor
				7 Leaf Node: 2 Class: virginica
			8 Feature: petal width (cm) Threshold: 1.5
				9 Leaf Node: 2 Class: virginica
				10 Feature: sepal length (cm) Threshold: 6.7
					11 Leaf Node: 1 Class: versicolor
					12 Leaf Node: 2 Class: virginica
		13 Feature: petal length (cm) Threshold: 4.8
			14 Feature: sepal length (cm) Threshold: 5.9
				15 Leaf Node: 1 Class: versicolor
				16 Leaf Node: 2 Class: virginica
			17 Leaf Node: 2 Class: virginica
